In [11]:
print(""" THIS TARIFF CHANGER OUTPUTS SDIs THAT MATCH AGREEMENTS TARIFF

Remaining issues with this version :: 
***Does support for OESL need to be added or can these be sent to LMSL processing?***
***Can we add error handling for to skip over these examples where MOP or DC doesn't comply with dictionary***""")

 THIS TARIFF CHANGER OUTPUTS SDIs THAT MATCH AGREEMENTS TARIFF

Remaining issues with this version :: 
***Does support for OESL need to be added or can these be sent to LMSL processing?***
***Can we add error handling for to skip over these examples where MOP or DC doesn't comply with dictionary***


In [2]:
# SQL Query
print("^ Pull Data from Query")

import pandas as pd
import tentaclio as tio

DB_CONN = 'databricks+thrift://hostname'
OUTPUT_PATH = '/Users/rob.cumpstey/Documents/Code'


query2 = """WITH max_record as (
                    SELECT meter_point_id, MAX(effective_from) as effective_from

                    FROM consumer.wh_electricity_estimated_annual_consumption_interval

                    --WHERE meter_point_id = 4551126

                    GROUP BY 1
),

eac_data as (
       SELECT MP.mpan,
       EAC_TABLE.estimated_annual_consumption as EAC,
       EAC_TABLE.time_pattern_regime_id as tpr,
       EAC_TABLE.meter_point_id, EAC_TABLE.effective_from as meter_point_id,
       EAC_TABLE.effective_from as effective_from
       

FROM consumer.wh_electricity_estimated_annual_consumption_interval AS EAC_TABLE

INNER JOIN consumer.stg_properties_electricitymeterpoint MP ON EAC_TABLE.meter_point_id = MP.meter_point_id
INNER JOIN max_record mr ON EAC_TABLE.meter_point_id = mr.meter_point_id
and mr.effective_from = EAC_TABLE.effective_from

WHERE EAC_TABLE.is_currently_active_record = TRUE
--and mpan IN ('1100010852148')
-- 1012521821007 // 1012721004456
-- another 2 rate example; 1100010852148

),

total_eac as ( SELECT mpan,
       sum(EAC) as acc_eac

FROM eac_data

/* WHERE mpan IN ('2000022938420','1419065901003','2200015044811','1900006327482','1900036162110','1012721004456','1800018420896','1100010852148','2312569361613',
'1417280351003','1591059350871','1012885420641','1300013340969','1012521821007','2000016666569','1012789644500','1100002865684','1419937571004','1012797120887',
'2600000668979') */

GROUP BY 1

)



SELECT DISTINCT
  MPOINT.mpan AS MPAN,
  grid_supply_point_id AS GSP,
  METER.meter_type AS Type,
  METER.serial_number AS MSN,
  marketparticipant.market_participant_name AS MOP,
  wh.supplier_id as SUPPLIER,
  te.acc_eac AS EAC,
  wh.tariff_code AS TARIFF,
  date(d.agreement_valid_from) AS agreement_start,
  date(DATEADD(DAY,1,date(d.agreement_valid_from))) as EFD,
  st.latest_dc_name as DC,
  MPOINT.supply_start_date AS SSD
  

FROM
  consumer.stg_properties_electricitymeterpoint AS MPOINT

  LEFT JOIN consumer.stg_properties_electricitymeter AS METER ON METER.meter_point_id = MPOINT.meter_point_id
  LEFT JOIN total_eac te ON MPOINT.mpan = te.mpan 
  LEFT JOIN consumer.stg_properties_electricityagentcontract AS AGENT on AGENT.meter_point_id = MPOINT.meter_point_id
  AND AGENT.contract_type = 'MOP'
  AND AGENT.agent_contract_effective_from <= CURRENT_DATE
  AND COALESCE(AGENT.agent_contract_effective_to, '9999-01-01') > CURRENT_DATE
  LEFT JOIN consumer.stg_electricity_marketparticipant AS marketparticipant ON marketparticipant.market_participant_id = AGENT.market_participant_id
  LEFT JOIN consumer.wh_electricity_validated_register_configuration_agreement_rate_interval wh ON MPOINT.mpan = wh.mpan
  LEFT JOIN consumer.wh_electricity_agreement_interval d ON wh.agreement_id = d.agreement_id
  LEFT JOIN consumer.fnl_industry_settlement st ON MPOINT.mpan = st.mpan

WHERE
    MPOINT.mpan IN ('2000022938420',
'1419065901003',
'2200015044811',
'1900006327482',
'1900036162110',
'1012721004456',
'1800018420896',
'1100010852148',
'2312569361613',
'1417280351003',
'1591059350871',
'1012885420641',
'1300013340969',
'1012521821007',
'2000016666569',
'1012789644500',
'1100002865684',
'1419937571004',
'1012797120887',
'2600000668979')
    AND METER.active_to IS NULL
    and wh.is_currently_active_agreement = TRUE AND wh.is_currently_active_register = TRUE AND d.is_currently_active_agreement = TRUE

GROUP BY 1, GSP, TYPE, MSN, MOP, SUPPLIER, EAC, TARIFF, agreement_start, DC, SSD;"""

with tio.db(DB_CONN) as client:
    df2 = client.get_df(query2)
display(df2)



^ Pull Data from Query


,MPAN,GSP,Type,MSN,MOP,SUPPLIER,EAC,TARIFF,agreement_start,EFD,DC,SSD
0,1300013340969,4,S2ADE,21E6006404,LEYLAND METERING SERVICES LTD,REGE,2298.0,E-2R-VAR-BB-23-04-01-D,2023-03-31,2023-04-01,ACCU,2022-05-26
1,2312569361613,12,S2ADE,19L3842293,LEYLAND METERING SERVICES LTD,REGE,21893.0,E-1R-VAR-BB-23-04-01-M,2023-06-12,2023-06-13,ACCU,2019-09-19
2,2000022938420,8,S1,Z16N426746,LEYLAND METERING SERVICES LTD,MRCY,1064.7,E-1R-VAR-22-11-01-H,2023-06-30,2023-07-01,UDMS,2021-02-12
3,2600000668979,13,S2A,20L3670209,LEYLAND METERING SERVICES LTD,MRCY,2389.7,E-FLAT2R-VAR-22-11-01-N,2023-03-31,2023-04-01,UDMS,2021-09-25
4,1100002865684,2,S1,13K0061586,LEYLAND METERING SERVICES LTD,MRCY,2470.8,E-2R-VAR-22-11-01-B,2023-10-25,2023-10-26,UDMS,2021-06-29
5,1012789644500,1,S2ADE,21L4033527,AES Smart Metering Ltd,REGE,2542.6,E-2R-LOYAL-FIX-12M-BB-23-12-30-A,2024-01-02,2024-01-03,ACCU,2020-09-14
6,2200015044811,11,S2ADE,21L3975255,OCTOPUS ENERGY SERVICES LTD,REGE,1829.9,E-1R-VAR-BB-23-04-01-L,2023-11-30,2023-12-01,ACCU,2019-05-16
7,1419937571004,5,S2ADE,22L4333843,LEYLAND METERING SERVICES LTD,MRCY,1128.0,E-FLAT2R-VAR-22-11-01-E,2023-03-31,2023-04-01,UDMS,2022-07-29
8,2000016666569,8,S2ADE,21L3274412,LEYLAND METERING SERVICES LTD,REGE,7006.3,E-2R-VAR-BB-23-04-01-H,2023-03-31,2023-04-01,ACCU,2019-04-19
9,1100010852148,2,S2ADE,21J0105977,OCTOPUS ENERGY SERVICES LTD,MRCY,1348.0,E-1R-VAR-22-11-01-B,2023-08-01,2023-08-02,UDMS,2021-08-02


In [3]:
# Export Data to CSV
print("^ Export Data to CSV")

df2.to_csv('/home/jovyan/shared/Settlement/SSC Mismatches Bulk/variables.csv')

^ Export Data to CSV


In [4]:
# date and time for titles, headers and footers
import os
from datetime import datetime as dt, timedelta
now = dt.now()
now_1 = now - timedelta(days=1)
now_2 = now - timedelta(days=2)
formatted_dt = now.strftime('%Y%m%d%H%M%S')
file_id_dt = now.strftime('%y%m%d')
file_id_dt_delay1 = now_1.strftime('%y%m%d')
file_id_dt_delay2 = now_2.strftime('%y%m%d')

print("^ Date + Time")

^ Date + Time


In [5]:
# ***** functions for writing d52 / d205 / d367 / cmex *****
print("^ Functions")

def generate_onerate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC, d52_body_output, d52_body_counter):
    onerate_d52_content = f'''121|{MPAN}|{formatted_SSD}|1|{formatted_EFD}|A|{formatted_EFD}|\n122|{SSC}|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00001|{EAC}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(onerate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_tworate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter):
    tworate_d52_content = f'''121|{MPAN}|{formatted_SSD}|2|{formatted_EFD}|A|{formatted_EFD}|\n122|{SSC}|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00043|{EAC_TPR_1}|{formatted_EFD}|\n124|00210|{EAC_TPR_2}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(tworate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_GSP_N_tworate_d52_content(MPAN, formatted_EFD, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter):
    tworate_d52_content = f'''121|{MPAN}|{formatted_SSD}|2|{formatted_EFD}|A|{formatted_EFD}|\n122|0722|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|13068|{EAC_TPR_1}|{formatted_EFD}|\n124|13067|{EAC_TPR_2}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(tworate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_GSP_P_onerate_d52_content(MPAN, formatted_EFD, GSP, EAC, d52_body_output, d52_body_counter):
    onerate_d52_content = f'''121|{MPAN}|{formatted_SSD}|1|{formatted_EFD}|A|{formatted_EFD}|\n122|0801|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00306|{EAC}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(onerate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1



def generate_d205_content(MPAN, formatted_EFD, SSC, MTC, PC, d205_body_ouput, d205_body_counter):
    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{formatted_SSD}|||||{MTC}|{formatted_EFD}|{PC}|{formatted_EFD}|{SSC}|{formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_GSP_P_d205_content(MPAN, formatted_EFD, d205_body_ouput, d205_body_counter):

    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{formatted_SSD}|||||801|{formatted_EFD}|1|{formatted_EFD}|0801|{formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_GSP_N_d205_content(MPAN, formatted_EFD, d205_body_ouput, d205_body_counter):
    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{formatted_SSD}|||||005|{formatted_EFD}|2|{formatted_EFD}|0722|{formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_onerate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    onerate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|{SSC}|{formatted_EFD}|\n03J|00001|\n04J|{MSN}|\n05J|1|Standard|1|C|AI|1.00|5|'''

    with open (d367_body_output, 'a') as body:
        body.write(onerate_d367_content)
        body.write('\n')

    d367_body_counter [0] += 1


def generate_tworate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    tworate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|{SSC}|{formatted_EFD}|\n03J|00043|\n04J|{MSN}|\n05J|1|Day|1|C|AI|1.00|5|\n03J|00210|\n04J|{MSN}|\n05J|2|Night|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(tworate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_tworate_GSP_N_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter):
    tworate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|0722|{formatted_EFD}|\n03J|13068|\n04J|{MSN}|\n05J|1|Day|1|C|AI|1.00|5|\n03J|13067|\n04J|{MSN}|\n05J|2|Night|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(tworate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_onerate_GSP_P_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    onerate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|0801|{formatted_EFD}|\n03J|00001|\n04J|{MSN}|\n05J|1|Standard|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(onerate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_onerate_cmex_content(MPAN, MSN, EFD, INITIAL1, INITIAL2, FINAL1):
    onerate_cmex_content = (f'''
            MPAN :: {MPAN} 
            MSN :: {MSN}
            Date of CMEX :: {EFD} 
            Initial Reads :: 1:{INITIAL1}   2:{INITIAL2}
            Final Read :: {FINAL1}''')
    return onerate_cmex_content


def generate_tworate_cmex_content(MPAN, MSN, EFD, INITIAL1, FINAL1, FINAL2):
    tworate_cmex_content = (f'''
            MPAN :: {MPAN}
            MSN :: {MSN}
            Date of CMEX :: {EFD}
            Initial Read :: 1:{INITIAL1}
            Final Read :: 1:{FINAL1}    2:{FINAL2}''')
    return tworate_cmex_content


def d52_headfoot_output(d52_filename, code, file_id_dt, formatted_dt, SUPPLIER, DC, d52_counter):
    with open (d52_filename, 'r') as count_d52:
        d52_lines = count_d52.readlines()
    
    total_d52_lines = len(d52_lines)
    
    with open(d52_filename, 'r') as d52_body:
        d52_existing_content = d52_body.read()
        
    d52_header = (f"ZHV|RC{file_id_dt}{code}|D0052001|X|{SUPPLIER}|D|{DC}|{formatted_dt}||||OPER|")
    d52_footer = (f"ZPT|RC{file_id_dt}{code}|{total_d52_lines}||{d52_counter[0]}|{formatted_dt}|")
    
    with open(d52_filename, 'w') as d52_headfoot:
        d52_headfoot.write(d52_header)
        d52_headfoot.write('\n')
        d52_headfoot.write(d52_existing_content)
        d52_headfoot.write(d52_footer)
        d52_headfoot.write('\n')
        

def d205_headfoot_output(d205_filename, d205_seq_number, DNO, formatted_dt, chosen_d205_counter, SUPPLIER):
    with open(d205_filename, 'r') as count_d205:
        d205_lines = count_d205.readlines()

    total_d205_lines = len(d205_lines)
    total_d205_lines += 1

    with open(d205_filename, 'r') as d205_body:
        d205_existing_content = d205_body.read()
        
    with open(d205_752_seq, 'r') as count_752:
        d205_seq_header = count_752.read()
        d205_seq_header = int(d205_seq_header)
        d205_seq_header += 1
        d205_seq_header = str(d205_seq_header)
        
    with open(d205_752_seq, 'w') as write_752:
        write_752.write(d205_seq_header)

    d205_header = f'ZHV|{d205_seq_number}|D0205001|X|{SUPPLIER}|P|{DNO}|{formatted_dt}||||OPER|\n752|{d205_seq_header}|'
    d205_footer = f'ZPT|{d205_seq_number}|{total_d205_lines}||1|{formatted_dt}|'

    with open(d205_filename, 'w') as d205_headfoot:
        d205_headfoot.write(d205_header)
        d205_headfoot.write('\n')
        d205_headfoot.write(d205_existing_content)
        d205_headfoot.write(d205_footer)
        

def d367_headfoot_output(d367_filename, d367_seq_number, SUPPLIER, MOP, formatted_dt, d367_body_counter):
    with open(d367_filename, 'r') as count_d367:
        d367_lines = count_d367.readlines()
    
    total_d367_lines = len(d367_lines)
    
    with open(d367_filename, 'r') as d367_body:
        d367_existing_content = d367_body.read()
        
    d367_header = (f"ZHV|{d367_seq_number}|D0367001|X|{SUPPLIER}|M|{MOP}|{formatted_dt}||||OPER|")
    d367_footer = (f"ZPT|{d367_seq_number}|{total_d367_lines}||{d367_body_counter}|{formatted_dt}|")

    with open(d367_filename, 'w') as d367_headfoot:
        d367_headfoot.write(d367_header)
        d367_headfoot.write('\n')
        d367_headfoot.write(d367_existing_content)
        d367_headfoot.write(d367_footer)


def delete_pre_script (directory):
    if os.path.exists(directory) and os.path.isdir(directory):
        outputs = os.listdir(directory)

        for file in outputs:
            file_path = os.path.join(directory, file)

            if os.path.isfile(file_path):
                os.remove(file_path)


def delete_post_script(counter_for_delete, file_for_delete):
    if counter_for_delete == [0]:
        if os.path.exists(file_for_delete):
            os.remove(file_for_delete)


def delete_post_script_d205(counter_for_delete, file_for_delete):
    if counter_for_delete == [1]:
        if os.path.exists(file_for_delete):
            os.remove(file_for_delete)

^ Functions


In [6]:
# ***** file directories *****
print("^ File Directories + Counters")

# universal
csv_filepath = '/home/jovyan/shared/Settlement/SSC Mismatches Bulk/variables.csv'
cmex_output = '/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/cmex_output.txt'
d205_752_seq = '/home/jovyan/shared/Settlement/SSC Mismatches Bulk/752.txt'

# mrcy only
mrcy_udms_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_UDMS_GR{file_id_dt}1b.D0052'
mrcy_accu_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_ACCU_GR{file_id_dt}2b.D0052'
mrcy_d205_EELC_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_EELC_GR{file_id_dt}0c.D0205'
mrcy_d205_EMEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_EMEB_GR{file_id_dt}1c.D0205'
mrcy_d205_LOND_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_LOND_GR{file_id_dt}2c.D0205'
mrcy_d205_MANW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_MANW_GR{file_id_dt}3c.D0205'
mrcy_d205_MIDE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_MIDE_GR{file_id_dt}4c.D0205'
mrcy_d205_NEEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_NEEB_GR{file_id_dt}5c.D0205'
mrcy_d205_NORW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_NORW_GR{file_id_dt}6c.D0205'
mrcy_d205_HYDE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_HYDE_GR{file_id_dt}7c.D0205'
mrcy_d205_SPOW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SPOW_GR{file_id_dt}8c.D0205'
mrcy_d205_SEEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SEEB_GR{file_id_dt}9c.D0205'
mrcy_d205_SOUT_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SOUT_GR{file_id_dt_delay1}0c.D0205'
mrcy_d205_SWAE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SWAE_GR{file_id_dt_delay1}1c.D0205'
mrcy_d205_SWEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SWEB_GR{file_id_dt_delay1}2c.D0205'
mrcy_d205_YELG_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_YELG_GR{file_id_dt_delay1}3c.D0205'
mrcy_d367_upld = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_UPLD_GR{file_id_dt}0g.D0367'
mrcy_d367_aesm = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_AESM_GR{file_id_dt}1g.D0367'
mrcy_d367_lmsl = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_LMSL_GR{file_id_dt}2g.D0367'
mrcy_d367_siem = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_SIEM_GR{file_id_dt}3g.D0367'
mrcy_d367_bmsl = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_BMSL_GR{file_id_dt}4g.D0367'
mrcy_d367_ukdc = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/MRCY_UKDC_GR{file_id_dt}5g.D0367'

# REGE only
rege_udms_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_UDMS_GR{file_id_dt}3b.D0052'
rege_accu_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_ACCU_GR{file_id_dt}4b.D0052'
rege_d205_EELC_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_EELC_GR{file_id_dt_delay1}4c.D0205'
rege_d205_EMEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_EMEB_GR{file_id_dt_delay1}5c.D0205'
rege_d205_LOND_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_LOND_GR{file_id_dt_delay1}6c.D0205'
rege_d205_MANW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_MANW_GR{file_id_dt_delay1}7c.D0205'
rege_d205_MIDE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_MIDE_GR{file_id_dt_delay1}8c.D0205'
rege_d205_NEEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_NEEB_GR{file_id_dt_delay1}9c.D0205'
rege_d205_NORW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_NORW_GR{file_id_dt_delay2}0c.D0205'
rege_d205_HYDE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_HYDE_GR{file_id_dt_delay2}1c.D0205'
rege_d205_SPOW_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SPOW_GR{file_id_dt_delay2}2c.D0205'
rege_d205_SEEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SEEB_GR{file_id_dt_delay2}3c.D0205'
rege_d205_SOUT_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SOUT_GR{file_id_dt_delay2}4c.D0205'
rege_d205_SWAE_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SWAE_GR{file_id_dt_delay2}5c.D0205'
rege_d205_SWEB_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SWEB_GR{file_id_dt_delay2}6c.D0205'
rege_d205_YELG_output = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_YELG_GR{file_id_dt_delay2}7c.D0205'
rege_d367_upld = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_UPLD_GR{file_id_dt}6g.D0367'
rege_d367_aesm = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_AESM_GR{file_id_dt}7g.D0367'
rege_d367_lmsl = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_LMSL_GR{file_id_dt}8g.D0367'
rege_d367_siem = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_SIEM_GR{file_id_dt}9g.D0367'
rege_d367_bmsl = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_BMSL_GR{file_id_dt_delay1}0g.D0367'
rege_d367_ukdc = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/REGE_UKDC_GR{file_id_dt_delay1}1g.D0367'

# delete old .txts
delete_cmex_52_367 = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/'
delete_d205s = f'/home/jovyan/shared/Settlement/SSC Mismatches Bulk/Outputs/'
delete_pre_script(directory=delete_cmex_52_367)
delete_pre_script(directory=delete_d205s)

# write new files
files_to_write = [mrcy_udms_output, mrcy_accu_output, mrcy_d205_EELC_output, mrcy_d205_EMEB_output,
                  mrcy_d205_LOND_output,
                  mrcy_d205_MANW_output, mrcy_d205_MIDE_output, mrcy_d205_NEEB_output, mrcy_d205_NORW_output,
                  mrcy_d205_HYDE_output, mrcy_d205_SPOW_output, mrcy_d205_SEEB_output, mrcy_d205_SOUT_output,
                  mrcy_d205_SWAE_output, mrcy_d205_SWEB_output, mrcy_d205_YELG_output, mrcy_d367_upld, mrcy_d367_aesm,
                  mrcy_d367_lmsl, mrcy_d367_siem, mrcy_d367_bmsl, mrcy_d367_ukdc,
                  rege_udms_output, rege_accu_output, rege_d205_EELC_output, rege_d205_EMEB_output,
                  rege_d205_LOND_output, rege_d205_MANW_output,
                  rege_d205_MIDE_output, rege_d205_NEEB_output, rege_d205_NORW_output, rege_d205_HYDE_output,
                  rege_d205_SPOW_output, rege_d205_SEEB_output, rege_d205_SOUT_output, rege_d205_SWAE_output,
                  rege_d205_SWEB_output, rege_d205_YELG_output, rege_d367_upld, rege_d367_aesm, rege_d367_lmsl,
                  rege_d367_siem, rege_d367_bmsl, rege_d367_ukdc,
                  cmex_output]
for files_to_write in files_to_write:
    open(files_to_write, 'x')

# ***** variables for flow header + footer *****
# counters for instances section of footers
mrcy_udms_counter = [0]
mrcy_accu_counter = [0]
mrcy_d205_EELC_counter = [1]
mrcy_d205_EMEB_counter = [1]
mrcy_d205_LOND_counter = [1]
mrcy_d205_MANW_counter = [1]
mrcy_d205_MIDE_counter = [1]
mrcy_d205_NEEB_counter = [1]
mrcy_d205_NORW_counter = [1]
mrcy_d205_HYDE_counter = [1]
mrcy_d205_SPOW_counter = [1]
mrcy_d205_SEEB_counter = [1]
mrcy_d205_SOUT_counter = [1]
mrcy_d205_SWAE_counter = [1]
mrcy_d205_SWEB_counter = [1]
mrcy_d205_YELG_counter = [1]
mrcy_upld_counter = [0]
mrcy_lmsl_counter = [0]
mrcy_aesm_counter = [0]
mrcy_siem_counter = [0]
mrcy_bmsl_counter = [0]
mrcy_ukdc_counter = [0]

rege_udms_counter = [0]
rege_accu_counter = [0]
rege_d205_EELC_counter = [1]
rege_d205_EMEB_counter = [1]
rege_d205_LOND_counter = [1]
rege_d205_MANW_counter = [1]
rege_d205_MIDE_counter = [1]
rege_d205_NEEB_counter = [1]
rege_d205_NORW_counter = [1]
rege_d205_HYDE_counter = [1]
rege_d205_SPOW_counter = [1]
rege_d205_SEEB_counter = [1]
rege_d205_SOUT_counter = [1]
rege_d205_SWAE_counter = [1]
rege_d205_SWEB_counter = [1]
rege_d205_YELG_counter = [1]
rege_upld_counter = [0]
rege_lmsl_counter = [0]
rege_aesm_counter = [0]
rege_siem_counter = [0]
rege_bmsl_counter = [0]
rege_ukdc_counter = [0]


^ File Directories + Counters


In [7]:
# dictionaries
print("^ Dictionaries")

# ***** d52 output dictionary *****
d52_dictionary = {
    'MRCY_UDMS': {'d52_body_output': mrcy_udms_output, 'd52_body_counter': mrcy_udms_counter},
    'MRCY_ACCU': {'d52_body_output': mrcy_accu_output, 'd52_body_counter': mrcy_accu_counter},
    'REGE_UDMS': {'d52_body_output': rege_udms_output, 'd52_body_counter': rege_udms_counter},
    'REGE_ACCU': {'d52_body_output': rege_accu_output, 'd52_body_counter': rege_accu_counter},
}

# ***** d205 output dictionary *****
d205_dictionary = {
    'MRCY_A': {'d205_body_output': mrcy_d205_EELC_output, 'd205_body_counter': mrcy_d205_EELC_counter},
    'MRCY_B': {'d205_body_output': mrcy_d205_EMEB_output, 'd205_body_counter': mrcy_d205_EMEB_counter},
    'MRCY_C': {'d205_body_output': mrcy_d205_LOND_output, 'd205_body_counter': mrcy_d205_LOND_counter},
    'MRCY_D': {'d205_body_output': mrcy_d205_MANW_output, 'd205_body_counter': mrcy_d205_MANW_counter},
    'MRCY_E': {'d205_body_output': mrcy_d205_MIDE_output, 'd205_body_counter': mrcy_d205_MIDE_counter},
    'MRCY_F': {'d205_body_output': mrcy_d205_NEEB_output, 'd205_body_counter': mrcy_d205_NEEB_counter},
    'MRCY_G': {'d205_body_output': mrcy_d205_NORW_output, 'd205_body_counter': mrcy_d205_NORW_counter},
    'MRCY_P': {'d205_body_output': mrcy_d205_HYDE_output, 'd205_body_counter': mrcy_d205_HYDE_counter},
    'MRCY_N': {'d205_body_output': mrcy_d205_SPOW_output, 'd205_body_counter': mrcy_d205_SPOW_counter},
    'MRCY_J': {'d205_body_output': mrcy_d205_SEEB_output, 'd205_body_counter': mrcy_d205_SEEB_counter},
    'MRCY_H': {'d205_body_output': mrcy_d205_SOUT_output, 'd205_body_counter': mrcy_d205_SOUT_counter},
    'MRCY_K': {'d205_body_output': mrcy_d205_SWAE_output, 'd205_body_counter': mrcy_d205_SWAE_counter},
    'MRCY_L': {'d205_body_output': mrcy_d205_SWEB_output, 'd205_body_counter': mrcy_d205_SWEB_counter},
    'MRCY_M': {'d205_body_output': mrcy_d205_YELG_output, 'd205_body_counter': mrcy_d205_YELG_counter},
    'REGE_A': {'d205_body_output': rege_d205_EELC_output, 'd205_body_counter': rege_d205_EELC_counter},
    'REGE_B': {'d205_body_output': rege_d205_EMEB_output, 'd205_body_counter': rege_d205_EMEB_counter},
    'REGE_C': {'d205_body_output': rege_d205_LOND_output, 'd205_body_counter': rege_d205_LOND_counter},
    'REGE_D': {'d205_body_output': rege_d205_MANW_output, 'd205_body_counter': rege_d205_MANW_counter},
    'REGE_E': {'d205_body_output': rege_d205_MIDE_output, 'd205_body_counter': rege_d205_MIDE_counter},
    'REGE_F': {'d205_body_output': rege_d205_NEEB_output, 'd205_body_counter': rege_d205_NEEB_counter},
    'REGE_G': {'d205_body_output': rege_d205_NORW_output, 'd205_body_counter': rege_d205_NORW_counter},
    'REGE_P': {'d205_body_output': rege_d205_HYDE_output, 'd205_body_counter': rege_d205_HYDE_counter},
    'REGE_N': {'d205_body_output': rege_d205_SPOW_output, 'd205_body_counter': rege_d205_SPOW_counter},
    'REGE_J': {'d205_body_output': rege_d205_SEEB_output, 'd205_body_counter': rege_d205_SEEB_counter},
    'REGE_H': {'d205_body_output': rege_d205_SOUT_output, 'd205_body_counter': rege_d205_SOUT_counter},
    'REGE_K': {'d205_body_output': rege_d205_SWAE_output, 'd205_body_counter': rege_d205_SWAE_counter},
    'REGE_L': {'d205_body_output': rege_d205_SWEB_output, 'd205_body_counter': rege_d205_SWEB_counter},
    'REGE_M': {'d205_body_output': rege_d205_YELG_output, 'd205_body_counter': rege_d205_YELG_counter},
}

d367_dictionary = {
    'MRCY_AESM': {'d367_body_output': mrcy_d367_aesm, 'd367_body_counter': mrcy_aesm_counter},
    'MRCY_LMSL': {'d367_body_output': mrcy_d367_lmsl, 'd367_body_counter': mrcy_lmsl_counter},
    'MRCY_UPLD': {'d367_body_output': mrcy_d367_upld, 'd367_body_counter': mrcy_upld_counter},
    'MRCY_SIEM': {'d367_body_output': mrcy_d367_siem, 'd367_body_counter': mrcy_siem_counter},
    'MRCY_BMSL': {'d367_body_output': mrcy_d367_bmsl, 'd367_body_counter': mrcy_bmsl_counter},
    'MRCY_UKDC': {'d367_body_output': mrcy_d367_ukdc, 'd367_body_counter': mrcy_ukdc_counter},
    'REGE_AESM': {'d367_body_output': rege_d367_aesm, 'd367_body_counter': rege_aesm_counter},
    'REGE_LMSL': {'d367_body_output': rege_d367_lmsl, 'd367_body_counter': rege_lmsl_counter},
    'REGE_UPLD': {'d367_body_output': rege_d367_upld, 'd367_body_counter': rege_upld_counter},
    'REGE_SIEM': {'d367_body_output': rege_d367_siem, 'd367_body_counter': rege_siem_counter},
    'REGE_BMSL': {'d367_body_output': rege_d367_bmsl, 'd367_body_counter': rege_bmsl_counter},
    'REGE_UKDC': {'d367_body_output': rege_d367_ukdc, 'd367_body_counter': rege_ukdc_counter},
}

^ Dictionaries


In [8]:
# ***** import data and define variables *****
print("^ Parse CSV + Define Variables + Write Flows")

# csv import
import csv
from datetime import datetime

with open(csv_filepath, 'r') as infile:
    reader = csv.reader(infile, delimiter=',')
    header = next(reader)

    for row in reader:
        MPAN = row[1]
        GSP = row[2]
        TYPE = row[3]
        MSN = row[4]
        MOP = row[5]
        SUPPLIER = row[6]
        EAC = row[7]
        EAC = float(EAC)
        EFD = row[10]
        TARIFF = row[8]
        DC = row[11]
        SSC = '9999'
        MTC = 000
        PC = 0
        DNO = row[2]
        SSD = row[12]
        #INITIAL1 = row[11]
        #INITIAL2 = row[12]
        #FINAL1 = row[13]
        #FINAL2 = row[14]
        EFD_datetime = datetime.strptime(EFD, '%Y-%m-%d')
        formatted_EFD = EFD_datetime.strftime('%Y%m%d')
        SSD_datetime = datetime.strptime(SSD, '%Y-%m-%d')
        formatted_SSD = SSD_datetime.strftime('%Y%m%d')
        

        # format desired SSC
        if "1R" in TARIFF:
            SSC = '0393'
        if "2R" in TARIFF :
            SSC = '0151'
        
        # conditional formatting for MTC
        if SSC == '0393':
            PC = 1
            MTC = 801

        if SSC == '0151':
            PC = 2
            MTC = 811

        # Conditional formatting for GSPs, DNOs defined in D205 arguments -
        if GSP == '1':
            GSP = 'A'
        if GSP == '2':
            GSP = 'B'
        if GSP == '3':
            GSP = 'C'
        if GSP == '4':
            GSP = 'D'
        if GSP == '5':
            GSP = 'E'
        if GSP == '6':
            GSP = 'F'
        if GSP == '7':
            GSP = 'G'
        if GSP == '8':
            GSP = 'H'
        if GSP == '9':
            GSP = 'J'
        if GSP == '10':
            GSP = 'K'
        if GSP == '11':
            GSP = 'L'
        if GSP == '12':
            GSP = 'M'
        if GSP == '13':
            GSP = 'N'
        if GSP == '14':
            GSP = 'P'

        # Conditional formatting for Northern Scotland 0801
        if GSP == '14':
            SSC = '0801'
        if GSP == '13':
            SSC = '0722'
            
        # MOP MPID Formatting
        if MOP == 'AES Smart Metering Ltd':
            MOP = 'AESM'
        if MOP == 'LEYLAND METERING SERVICES LTD':
            MOP = 'LMSL'
        if MOP == 'OCTOPUS ENERGY SERVICES LTD':
            MOP = 'LMSL'
        if MOP == 'SMS Energy Services Ltd':
            MOP = 'UPLD'
        if MOP == 'Siemens MCS':
            MOP = 'SIEM'
        if MOP == 'BURY METERING SERVICES LTD':
            MOP = 'BMSL'
        if MOP == 'IMServ Europe Ltd':
            MOP = 'UKDC'
            
            
            
            
            
        # ***** write d52 / d205 / d367 / cmex *****
        # write d52 + EAC Edit
        if SSC == '0151':
            EAC_TPR_1 = round(EAC * 0.667, 1)
            EAC_TPR_2 = round(EAC * 0.333, 1)

        d52_key = f'{SUPPLIER}_{DC}'

        if d52_key in d52_dictionary:
            d52_dict_output = d52_dictionary[d52_key]
        d52_body_output = d52_dict_output['d52_body_output']
        d52_body_counter = d52_dict_output['d52_body_counter']

        if GSP == 'P' and SSC == '0393':
            generate_GSP_P_onerate_d52_content(MPAN, formatted_EFD, GSP, EAC, d52_body_output, d52_body_counter)

        if GSP != 'P' and SSC == '0393':
            generate_onerate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC, d52_body_output, d52_body_counter)

        if GSP == 'N' and SSC == '0151':
            generate_GSP_N_tworate_d52_content(MPAN, formatted_EFD, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter)

        if GSP !='N' and SSC == '0151':
            generate_tworate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter)



        # write d205
        d205_key = f'{SUPPLIER}_{GSP}'

        if d205_key in d205_dictionary:
            d205_dict_output = d205_dictionary[d205_key]
        d205_body_output = d205_dict_output['d205_body_output']
        d205_body_counter = d205_dict_output['d205_body_counter']

        if GSP == 'P' and SSC == '0393':
            generate_GSP_P_d205_content(MPAN, formatted_EFD, 'd205_body_ouput', d205_body_counter)
            
        if GSP == 'P' and SSC == '0151':
            generate_d205_content(MPAN, formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)

        if GSP == 'N' and SSC == '0151':
            generate_GSP_N_d205_content(MPAN, formatted_EFD, d205_body_output, d205_body_counter)
            
        if GSP == 'N' and SSC == '0393':
            generate_d205_content(MPAN, formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)

        if GSP != 'P' and GSP != 'N':
            generate_d205_content(MPAN, formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)


        # write d367
        d367_key = f'{SUPPLIER}_{MOP}'

        if d367_key in d367_dictionary:
            d367_dict_output = d367_dictionary[d367_key]
        d367_body_output = d367_dict_output['d367_body_output']
        d367_body_counter = d367_dict_output['d367_body_counter']

        if GSP == 'N' and SSC == '0151':
            generate_tworate_GSP_N_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter)

        if GSP !='N' and SSC == '0151':
            generate_tworate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter)

        if SSC == '0393' and GSP == 'P':
            generate_onerate_GSP_P_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter)

        if GSP !='P' and SSC == '0393':
            generate_onerate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter)


        # write cmex
        #onerate_cmex_content = generate_onerate_cmex_content(MPAN, MSN, EFD, INITIAL1, INITIAL2, FINAL1)
        #tworate_cmex_content = generate_tworate_cmex_content(MPAN, MSN, EFD, INITIAL1, FINAL1, FINAL2)

        #if SSC == '0393':
            #with open(cmex_output, 'a') as i:
                #i.write(onerate_cmex_content)
                #i.write('\n')
                #i.write('\n')

        #if SSC == '0151':
            #with open(cmex_output, 'a') as i2:
                #i2.write(tworate_cmex_content)
                #i2.write('\n')
                #i2.write('\n')

^ Parse CSV + Define Variables + Write Flows


In [9]:
# ***** write header + footers *****
print("^ Write Headers + Footers")

# cmex header
cmex_header_content = (f'Can you please process the following CMEXs for this weeks SSC changes')

with open(cmex_output, 'r') as cmexr:
    cmex_existing_content = cmexr.read()

with open(cmex_output, 'w') as cmexw:
    cmexw.write(cmex_header_content)
    cmexw.write('\n')
    cmexw.write('\n')
    cmexw.write(cmex_existing_content)
    cmexw.write('\n')

# d52 header + footer
# rege_udms
d52_headfoot_output(d52_filename=mrcy_udms_output, code='2a', file_id_dt=file_id_dt, formatted_dt=formatted_dt, SUPPLIER='MRCY', d52_counter=mrcy_udms_counter, DC='UDMS')
d52_headfoot_output(d52_filename=mrcy_accu_output, code='2b', file_id_dt=file_id_dt, formatted_dt=formatted_dt, SUPPLIER='MRCY', d52_counter=mrcy_accu_counter, DC='ACCU')
d52_headfoot_output(d52_filename=rege_udms_output, code='2c', file_id_dt=file_id_dt, formatted_dt=formatted_dt, SUPPLIER='REGE', d52_counter=rege_udms_counter, DC='UDMS')
d52_headfoot_output(d52_filename=rege_accu_output, code='2d', file_id_dt=file_id_dt, formatted_dt=formatted_dt, SUPPLIER='REGE', d52_counter=rege_accu_counter, DC='ACCU')


    # d205 header + footer shortcode
    # rege
d205_headfoot_output(d205_filename=mrcy_d205_EELC_output, chosen_d205_counter=mrcy_d205_EELC_counter[0],
                              DNO='EELC', d205_seq_number=f'GR{file_id_dt}0c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_EMEB_output, chosen_d205_counter=mrcy_d205_EMEB_counter[0],
                              DNO='EMEB', d205_seq_number=f'GR{file_id_dt}1c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_LOND_output, chosen_d205_counter=mrcy_d205_LOND_counter[0],
                              DNO='LOND', d205_seq_number=f'GR{file_id_dt}2c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_MANW_output, chosen_d205_counter=mrcy_d205_MANW_counter[0],
                              DNO='MANW', d205_seq_number=f'GR{file_id_dt}3c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_MIDE_output, chosen_d205_counter=mrcy_d205_MIDE_counter[0],
                              DNO='MIDE', d205_seq_number=f'GR{file_id_dt}4c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_NEEB_output, chosen_d205_counter=mrcy_d205_NEEB_counter[0],
                              DNO='NEEB', d205_seq_number=f'GR{file_id_dt}5c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_NORW_output, chosen_d205_counter=mrcy_d205_NORW_counter[0],
                              DNO='NORW', d205_seq_number=f'GR{file_id_dt}6c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_HYDE_output, chosen_d205_counter=mrcy_d205_HYDE_counter[0],
                              DNO='HYDE', d205_seq_number=f'GR{file_id_dt}7c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_SPOW_output, chosen_d205_counter=mrcy_d205_SPOW_counter[0],
                              DNO='SPOW', d205_seq_number=f'GR{file_id_dt}8c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_SEEB_output, chosen_d205_counter=mrcy_d205_SEEB_counter[0],
                              DNO='SEEB', d205_seq_number=f'GR{file_id_dt}9c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_SOUT_output, chosen_d205_counter=mrcy_d205_SOUT_counter[0],
                              DNO='SOUT', d205_seq_number=f'GR{file_id_dt_delay1}0c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_SWAE_output, chosen_d205_counter=mrcy_d205_SWAE_counter[0],
                              DNO='SWAE', d205_seq_number=f'GR{file_id_dt_delay1}1c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_SWEB_output, chosen_d205_counter=mrcy_d205_SWEB_counter[0],
                              DNO='SWEB', d205_seq_number=f'GR{file_id_dt_delay1}2c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
d205_headfoot_output(d205_filename=mrcy_d205_YELG_output, chosen_d205_counter=mrcy_d205_YELG_counter[0],
                              DNO='YELG', d205_seq_number=f'GR{file_id_dt_delay1}3c', formatted_dt=formatted_dt, SUPPLIER='MRCY')
    # rege
d205_headfoot_output(d205_filename=rege_d205_EELC_output, chosen_d205_counter=rege_d205_EELC_counter[0],
                              DNO='EELC', d205_seq_number=f'GR{file_id_dt_delay1}4c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_EMEB_output, chosen_d205_counter=rege_d205_EMEB_counter[0],
                              DNO='EMEB', d205_seq_number=f'GR{file_id_dt_delay1}5c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_LOND_output, chosen_d205_counter=rege_d205_LOND_counter[0],
                              DNO='LOND', d205_seq_number=f'GR{file_id_dt_delay1}6c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_MANW_output, chosen_d205_counter=rege_d205_MANW_counter[0],
                              DNO='MANW', d205_seq_number=f'GR{file_id_dt_delay1}7c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_MIDE_output, chosen_d205_counter=rege_d205_MIDE_counter[0],
                              DNO='MIDE', d205_seq_number=f'GR{file_id_dt_delay1}8c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_NEEB_output, chosen_d205_counter=rege_d205_NEEB_counter[0],
                              DNO='NEEB', d205_seq_number=f'GR{file_id_dt_delay1}9c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_NORW_output, chosen_d205_counter=rege_d205_NORW_counter[0],
                              DNO='NORW', d205_seq_number=f'GR{file_id_dt_delay2}0c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_HYDE_output, chosen_d205_counter=rege_d205_HYDE_counter[0],
                              DNO='HYDE', d205_seq_number=f'GR{file_id_dt_delay2}1c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_SPOW_output, chosen_d205_counter=rege_d205_SPOW_counter[0],
                              DNO='SPOW', d205_seq_number=f'GR{file_id_dt_delay2}2c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_SEEB_output, chosen_d205_counter=rege_d205_SEEB_counter[0],
                              DNO='SEEB', d205_seq_number=f'GR{file_id_dt_delay2}3c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_SOUT_output, chosen_d205_counter=rege_d205_SOUT_counter[0],
                              DNO='SOUT', d205_seq_number=f'GR{file_id_dt_delay2}4c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_SWAE_output, chosen_d205_counter=rege_d205_SWAE_counter[0],
                              DNO='SWAE', d205_seq_number=f'GR{file_id_dt_delay2}5c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_SWEB_output, chosen_d205_counter=rege_d205_SWEB_counter[0],
                              DNO='SWEB', d205_seq_number=f'GR{file_id_dt_delay2}6c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')
d205_headfoot_output(d205_filename=rege_d205_YELG_output, chosen_d205_counter=rege_d205_YELG_counter[0],
                              DNO='YELG', d205_seq_number=f'GR{file_id_dt_delay2}7c', formatted_dt=formatted_dt,
                              SUPPLIER='REGE')

# D367 header + footer
# mrcy
d367_headfoot_output(d367_filename=mrcy_d367_aesm, d367_seq_number=f'GR{file_id_dt}1g', SUPPLIER='MRCY', MOP='AESM', formatted_dt=formatted_dt, d367_body_counter=mrcy_aesm_counter[0])
d367_headfoot_output(d367_filename=mrcy_d367_lmsl, d367_seq_number=f'GR{file_id_dt}2g', SUPPLIER='MRCY', MOP='LMSL', formatted_dt=formatted_dt, d367_body_counter=mrcy_lmsl_counter[0])
d367_headfoot_output(d367_filename=mrcy_d367_upld, d367_seq_number=f'GR{file_id_dt}0g', SUPPLIER='MRCY', MOP='UPLD', formatted_dt=formatted_dt, d367_body_counter=mrcy_upld_counter[0])
d367_headfoot_output(d367_filename=mrcy_d367_siem, d367_seq_number=f'GR{file_id_dt}3g', SUPPLIER='MRCY', MOP='SIEM', formatted_dt=formatted_dt, d367_body_counter=mrcy_siem_counter[0])
d367_headfoot_output(d367_filename=mrcy_d367_bmsl, d367_seq_number=f'GR{file_id_dt}4g', SUPPLIER='MRCY', MOP='BMSL', formatted_dt=formatted_dt, d367_body_counter=mrcy_bmsl_counter[0])
d367_headfoot_output(d367_filename=mrcy_d367_ukdc, d367_seq_number=f'GR{file_id_dt}5g', SUPPLIER='MRCY', MOP='UKDC', formatted_dt=formatted_dt, d367_body_counter=mrcy_ukdc_counter[0])
#rege
d367_headfoot_output(d367_filename=rege_d367_aesm, d367_seq_number=f'GR{file_id_dt}7g', SUPPLIER='REGE', MOP='AESM', formatted_dt=formatted_dt, d367_body_counter=rege_aesm_counter[0])
d367_headfoot_output(d367_filename=rege_d367_lmsl, d367_seq_number=f'GR{file_id_dt}8g', SUPPLIER='REGE', MOP='LMSL', formatted_dt=formatted_dt, d367_body_counter=rege_lmsl_counter[0])
d367_headfoot_output(d367_filename=rege_d367_upld, d367_seq_number=f'GR{file_id_dt}6g', SUPPLIER='REGE', MOP='UPLD', formatted_dt=formatted_dt, d367_body_counter=rege_upld_counter[0])
d367_headfoot_output(d367_filename=rege_d367_siem, d367_seq_number=f'GR{file_id_dt}9g', SUPPLIER='REGE', MOP='SIEM', formatted_dt=formatted_dt, d367_body_counter=rege_siem_counter[0])
d367_headfoot_output(d367_filename=rege_d367_bmsl, d367_seq_number=f'GR{file_id_dt_delay1}0g', SUPPLIER='REGE', MOP='BMSL', formatted_dt=formatted_dt, d367_body_counter=rege_bmsl_counter[0])
d367_headfoot_output(d367_filename=rege_d367_ukdc, d367_seq_number=f'GR{file_id_dt_delay1}1g', SUPPLIER='REGE', MOP='UKDC', formatted_dt=formatted_dt, d367_body_counter=rege_ukdc_counter[0])


^ Write Headers + Footers


In [10]:
# ***** delete unused .txts *****
print("^ Post Script Cleanup")

# mrcy
delete_post_script(counter_for_delete=mrcy_udms_counter, file_for_delete=mrcy_udms_output)
delete_post_script(counter_for_delete=mrcy_accu_counter, file_for_delete=mrcy_accu_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_EELC_counter, file_for_delete=mrcy_d205_EELC_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_EMEB_counter, file_for_delete=mrcy_d205_EMEB_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_LOND_counter, file_for_delete=mrcy_d205_LOND_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_MANW_counter, file_for_delete=mrcy_d205_MANW_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_MIDE_counter, file_for_delete=mrcy_d205_MIDE_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_NEEB_counter, file_for_delete=mrcy_d205_NEEB_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_NORW_counter, file_for_delete=mrcy_d205_NORW_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_HYDE_counter, file_for_delete=mrcy_d205_HYDE_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_SPOW_counter, file_for_delete=mrcy_d205_SPOW_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_SEEB_counter, file_for_delete=mrcy_d205_SEEB_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_SOUT_counter, file_for_delete=mrcy_d205_SOUT_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_SWAE_counter, file_for_delete=mrcy_d205_SWAE_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_SWEB_counter, file_for_delete=mrcy_d205_SWEB_output)
delete_post_script_d205(counter_for_delete=mrcy_d205_YELG_counter, file_for_delete=mrcy_d205_YELG_output)
delete_post_script(counter_for_delete=mrcy_upld_counter, file_for_delete=mrcy_d367_upld)
delete_post_script(counter_for_delete=mrcy_aesm_counter, file_for_delete=mrcy_d367_aesm)
delete_post_script(counter_for_delete=mrcy_lmsl_counter, file_for_delete=mrcy_d367_lmsl)
delete_post_script(counter_for_delete=mrcy_siem_counter, file_for_delete=mrcy_d367_siem)
delete_post_script(counter_for_delete=mrcy_bmsl_counter, file_for_delete=mrcy_d367_bmsl)
delete_post_script(counter_for_delete=mrcy_ukdc_counter, file_for_delete=mrcy_d367_ukdc)
# rege
delete_post_script(counter_for_delete=rege_udms_counter, file_for_delete=rege_udms_output)
delete_post_script(counter_for_delete=rege_accu_counter, file_for_delete=rege_accu_output)
delete_post_script_d205(counter_for_delete=rege_d205_EELC_counter, file_for_delete=rege_d205_EELC_output)
delete_post_script_d205(counter_for_delete=rege_d205_EMEB_counter, file_for_delete=rege_d205_EMEB_output)
delete_post_script_d205(counter_for_delete=rege_d205_LOND_counter, file_for_delete=rege_d205_LOND_output)
delete_post_script_d205(counter_for_delete=rege_d205_MANW_counter, file_for_delete=rege_d205_MANW_output)
delete_post_script_d205(counter_for_delete=rege_d205_MIDE_counter, file_for_delete=rege_d205_MIDE_output)
delete_post_script_d205(counter_for_delete=rege_d205_NEEB_counter, file_for_delete=rege_d205_NEEB_output)
delete_post_script_d205(counter_for_delete=rege_d205_NORW_counter, file_for_delete=rege_d205_NORW_output)
delete_post_script_d205(counter_for_delete=rege_d205_HYDE_counter, file_for_delete=rege_d205_HYDE_output)
delete_post_script_d205(counter_for_delete=rege_d205_SPOW_counter, file_for_delete=rege_d205_SPOW_output)
delete_post_script_d205(counter_for_delete=rege_d205_SEEB_counter, file_for_delete=rege_d205_SEEB_output)
delete_post_script_d205(counter_for_delete=rege_d205_SOUT_counter, file_for_delete=rege_d205_SOUT_output)
delete_post_script_d205(counter_for_delete=rege_d205_SWAE_counter, file_for_delete=rege_d205_SWAE_output)
delete_post_script_d205(counter_for_delete=rege_d205_SWEB_counter, file_for_delete=rege_d205_SWEB_output)
delete_post_script_d205(counter_for_delete=rege_d205_YELG_counter, file_for_delete=rege_d205_YELG_output)
delete_post_script(counter_for_delete=rege_upld_counter, file_for_delete=rege_d367_upld)
delete_post_script(counter_for_delete=rege_aesm_counter, file_for_delete=rege_d367_aesm)
delete_post_script(counter_for_delete=rege_lmsl_counter, file_for_delete=rege_d367_lmsl)
delete_post_script(counter_for_delete=rege_siem_counter, file_for_delete=rege_d367_siem)
delete_post_script(counter_for_delete=rege_bmsl_counter, file_for_delete=rege_d367_bmsl)
delete_post_script(counter_for_delete=rege_ukdc_counter, file_for_delete=rege_d367_ukdc)

^ Post Script Cleanup
